# __AAL Calculator__ 

## Benchmark Notebook

In [1]:
# User Parameters
prob_mod_core_path = r'C:\Users\slawler\GitRepos\probmod-tools\{}'

In [2]:
wse_file = "s3://pfra/RiskAssessment/Sacramento/Results/F01/WSE_Sacramento_F01_Uniform.csv"
weights_path = ""
curve_groups = {"singFam_1Story_NoBasement": [105, 129, 132, 139, 173], "singFam_2Story_NoBasement": [107, 130, 136, 140, 174], "singFam_3Story_NoBasement": [109], "singFam_1Story_Basement": [106, 121, 133, 181, 704], "singFam_2Story_Basement": [108, 122, 137], "singFam_3Story_Basement": [110, 123], "mobileHome": [189, 191, 192, 203]}
structure_cols = {"Unique Building ID": "plus_code", "Damage Code": "damage_code", "Building Limit": "bldg_limit", "Building Deduction": "bldg_ded", "Content Limit": "cnt_limit", "Content Deduction": "cnt_ded", "Ground Elevation": "GroundElev", "First Floor Height": "first_floor_elev"}

scen = "Sacramento_F01_Uniform"
TRI = ""

import sys
import pathlib as pl
import pandas as pd
from time import time
from scipy.interpolate import interp1d
from multiprocessing import Pool, cpu_count

root = pl.Path(prob_mod_core_path.format('risk'))
sys.path.append(prob_mod_core_path.format('core'))
from risk_refactor import *

project = scen.split('_')[0]
model_name = scen.split('_')[1]
book = scen.split('_')[2]
if model_name[0] == 'P':
    pluvial = True
else:
    pluvial = False
    
defaultHazusDDFn_path = root / 'hazusdepthdmgfns' / 'Building_DDF_Full_LUT_Hazus3p0.json'
df_BDDFn = pd.read_json(str(defaultHazusDDFn_path), orient = 'index')
df_BDDFn = hazusID_to_depth(df_BDDFn)
df_agg = aggregate_ddf_curves(df_BDDFn, curve_groups, plot=False)

# set curve for single family 1 story no basement
df_agg.loc[-1, 'singFam_1Story_NoBasement']= 0.6
df_agg.loc[-0.000000000001, 'singFam_1Story_NoBasement']= 0.6

# set curve for single family 2 story no basement
df_agg.loc[-1, 'singFam_2Story_NoBasement']= 0.6
df_agg.loc[-0.000000000001, 'singFam_2Story_NoBasement']= 0.6

# set curve for single family 3 story no basement
df_agg.loc[-1, 'singFam_3Story_NoBasement']= 0.0
df_agg.loc[-0.000000000001, 'singFam_3Story_NoBasement']= 0.0

# set curve for mobile home
df_agg.loc[-0.000000000001, 'mobileHome']= 0.75

df_agg = df_agg.sort_index()

#wse_file = 's3://pfra/RiskAssessment/{0}/Results/{1}/WSE_{0}_{1}_{2}.csv'.format(project, model_name, book)
breach_prob_file = 's3://pfra/RiskAssessment/{0}/BreachAnalysis/{0}_{1}_raw_prob_table.csv'.format(project, model_name)
weights_file = 's3://pfra/RiskAssessment/{0}/BreachAnalysis/{0}_{1}.xlsx'.format(project, model_name)

dfwse = pd.read_csv(wse_file, index_col='plus_code')
dfw = pd.read_excel(weights_file, sheet_name='Event_Weights', index_col=0 )[['Overall Weight']]
dfbp = pd.read_csv(breach_prob_file, sep='\t', index_col=0)
for col in dfbp.columns:
    dfbp.rename(columns={col:col.split('_')[1]}, inplace=True)
    
wse_results = [c for c in dfwse.columns if '_E' in c]
thin_df = dfwse[wse_results].copy()

thin_df['Max'] = thin_df.max(axis=1)
non_zeros = thin_df[thin_df['Max'] > 0].index
print('Non-Zeros Points', len(non_zeros))

# Slice the full wsedf to select only plus codes with data to compute
compute_matrix = dfwse.loc[non_zeros].drop(columns='geom').copy()

Non-Zeros Points 182935


### Grab an data point for testing

In [3]:
pcodes = compute_matrix.index[0:1]
compute_chunk = compute_matrix.loc[pcodes].copy()

### Create global_model_data boject & get arguments for main function

In [4]:
global_model_data = FluvialEvents(dfw, dfbp)
allargs = [(p_code, global_model_data, df_agg, compute_chunk.loc[p_code], structure_cols) for p_code in compute_chunk.index]

In [6]:
test_point = allargs[0]
result = calc_fluv_aal_mp_functions(test_point)
result

('84CW295C+RG47G34', 20210.438716750774)

#### Easy reloader if changing risk_refactor

In [ ]:
from importlib import reload
import risk_refactor
reload(risk_refactor)
from risk_refactor import *